In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pyshark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
from progressbar import *  

In [ ]:
cap = pyshark.FileCapture("p1p1.17.ip.56.pcap")

# 服务数量统计

In [ ]:
packet_num = 0
record = []

In [ ]:
def find_services(packet):
    global packet_num
    packet_num += 1
    if int(packet.tcp.flags_syn) == 1 and int(packet.tcp.flags_ack) == 0:
        service = str(packet.ip.dst) + ":" + str(packet.tcp.dstport)
        if service in record:
            pass
        else:
            record.append(service)
            print "New service found:", service

In [ ]:
cap.apply_on_packets(find_services)

# TCP请求统计

In [ ]:
services = {}
for service in record:
    services[service] = {}
    services[service]["syn"] = 1
    services[service]["links"] = {}
    services[service]["complete_link"] = 0
    services[service]["half_link"] = 0

In [ ]:
widgets = ['Processing: ', Percentage(), ' ', Bar(marker='#',left='[',right=']'),
           ' ', ETA()] 

pbar = ProgressBar(widgets=widgets, maxval=packet_num+1)
pbar.start()

count = 0

for pkt in cap:
    
    count += 1

    src = str(pkt.ip.src) + ":" + str(pkt.tcp.srcport)
    dst = str(pkt.ip.dst) + ":" + str(pkt.tcp.dstport)

    if (int(pkt.tcp.flags_syn) == 1) and (int(pkt.tcp.flags_ack) == 0):
        if dst in services.keys():
            services[dst]["syn"] += 1
            if (src in services[dst]["links"]):
                if (services[dst]["links"][src]):
                    services[dst]["half_link"] += 1
            services[dst]["links"][src] = False

    if (int(pkt.tcp.flags_syn) == 1) and (int(pkt.tcp.flags_ack) == 1):
        services[src]["links"][dst] = True

    if (int(pkt.tcp.flags_syn) == 0) and (int(pkt.tcp.flags_ack) == 1):
        if (dst in services.keys()):
            if (src in services[dst]["links"]):
                if (services[dst]["links"][src]):
                    services[dst]["complete_link"] += 1
                    del services[dst]["links"][src]
                    
pbar.finish()

In [ ]:
for service in services:
    print "Service:", service
    print "#Syn:", services[service]["syn"]
    print "#TCP-Connection:", services[service]["complete_link"]
    print "#Half-Connection:", str(len(services[service]["links"]) + services[service]["half_link"])  

# 重传率统计

In [ ]:
services = {}
for service in record:
    services[service] = {}
    services[service]["retrans"] = 0
    services[service]["total"] = 0    

In [ ]:
widgets = ['Processing: ', Percentage(), ' ', Bar(marker='#',left='[',right=']'),
           ' ', ETA()] 

pbar = ProgressBar(widgets=widgets, maxval=packet_num+1)
pbar.start()

count = 0
for pkt in cap:
    
    count += 1

    src = str(pkt.ip.src) + ":" + str(pkt.tcp.srcport)
    dst = str(pkt.ip.dst) + ":" + str(pkt.tcp.dstport)

    if (pkt.tcp.get('analysis_retransmission')):
        if src in services.keys():
            services[src]["retrans"] += 1
        else:
            services[dst]["retrans"] += 1
            
    if (src in services.keys()):
        services[src]["total"] += 1
    else:
        services[dst]["total"] += 1

pbar.finish()

In [ ]:
for service in record:
    print "Service:", service
    print "#Retransmission", services[service]["retrans"]
    print "Retransmission %:", float(services[service]["retrans"]) / services[service]["total"]

# TCP链接重置请求统计

In [ ]:
services = {}
for service in record:
    services[service] = {}
    services[service]["reset_client"] = 0
    services[service]["reset_server"] = 0

In [ ]:
widgets = ['Processing: ', Percentage(), ' ', Bar(marker='#',left='[',right=']'),
           ' ', ETA()] 

pbar = ProgressBar(widgets=widgets, maxval=packet_num+1)
pbar.start()

count = 0

for pkt in cap:
    
    count += 1

    src = str(pkt.ip.src) + ":" + str(pkt.tcp.srcport)
    dst = str(pkt.ip.dst) + ":" + str(pkt.tcp.dstport)

    if (int(pkt.tcp.flags_reset) == 1):
        if src in services.keys():
            services[src]["reset_server"] += 1
        else:
            services[dst]["reset_client"] += 1
            
pbar.finish()

In [ ]:
for service in record:
    print "Service:", service
    print "#Reset:Client", services[service]["reset_client"]
    print "#Reset:Server", services[service]["reset_server"]

# 流量统计

In [ ]:
flow = []

In [ ]:
widgets = ['Processing: ', Percentage(), ' ', Bar(marker='#',left='[',right=']'),
           ' ', ETA()] 

pbar = ProgressBar(widgets=widgets, maxval=packet_num+1)
pbar.start()

count = 0

for pkt in cap:
    flow.append((pkt.sniff_time, pkt.length))
    count += 1
    pbar.update(count)
    
pbar.finish()

In [ ]:
flow = pd.Series(index=map(lambda x: x[0], flow), data=map(lambda x: float(x[1]), flow))

In [ ]:
stat = flow.resample("300s").sum()

In [ ]:
plt.figure(figsize=(16, 5))
plt.plot(stat)
plt.plot(stat, 'o', color="purple")